<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

In [26]:
#your code goes here
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import desc
from pyspark.sql.functions import col

In [ ]:
# Start session

In [4]:
spark = SparkSession.builder.appName("Capstone MLOps").getOrCreate()

24/04/08 18:06:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2024-04-08 18:06:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2024-04-08 18:06:46 (46.1 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [ ]:
# Load the csv into a spark dataframe

In [65]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [20]:
print(f"Rows: {df.count()}, columns: {len(df.columns)}")

Rows: 10000, columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [10]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [24]:
df.dtypes[3]

('searchterm', 'string')

In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [28]:
df.filter(col("searchterm") == "gaming laptop").count()

499

In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [18]:
termsCount = df.groupBy("searchterm").count()
termsCount = termsCount.sort(desc("count"))
termsCount.show(5)

[Stage 4:===================================================>   (189 + 9) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [42]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2024-04-08 18:41:19--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-04-08 18:41:19 (6.86 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [43]:
import tarfile
tar_gz_file = "model.tar.gz"

# Extraer el contenido del archivo tar.gz
with tarfile.open(tar_gz_file, "r:gz") as tar:
    tar.extractall()

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [49]:
from pyspark.ml.regression import LinearRegressionModel
forecastModel = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [71]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','year')
    predictions = forecastModel.transform(__)
    predictions.select('prediction').show()

In [72]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/04/08 19:30:10 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/04/08 19:30:10 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
